# Import Dependencies

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
import optuna
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.layers import Dropout
from keras import regularizers
from keras import optimizers
#from keras import dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Parameters

In [2]:
# Set the Seed
randomSeed = 1000
pd.set_option('display.max_columns', None)

# Neural Network

In [ ]:
# Splitting the dataset into the Training set and Test set
X_train, X_test, y_train, y_test = train_test_split(X3, y3, test_size = 0.2, random_state = randomSeed)

# Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [37]:
X_train.shape

(10882, 59)

In [ ]:

# Initialising the ANN
model = Sequential()

# Adding the input layer and the first hidden layer
model.add(Dense(32, activation = 'relu', input_dim = 59))

# Adding the second hidden layer
model.add(Dense(units = 32, activation = 'relu', activity_regularizer=regularizers.l2(0.01), 
                kernel_regularizer=regularizers.l2(0.01)))

# Add a dropout layer
model.add(Dropout(0.1))

# Adding the third hidden layer
model.add(Dense(units = 32, activation = 'relu', activity_regularizer=regularizers.l2(0.01), 
                kernel_regularizer=regularizers.l2(0.01)))

# Adding the output layer
model.add(Dense(units = 1))

#best: lr= 0.01, batch_size = 10, epochs = 30, one dropout layer between 2 and 3
# Compiling the ANN
model.compile(optimizer = optimizers.Adam(learning_rate=0.01), loss = 'mean_squared_error')

# Fitting the ANN to the Training set
model.fit(X_train, y_train, batch_size = 10, epochs = 40)

y_pred = model.predict(X_test)


In [116]:
#convert y_pred to 1D array
y_pred = y_pred.flatten()
np.sqrt(np.mean((y_test - y_pred)**2))


45480.466272972415

CV for NN

In [ ]:
sc = StandardScaler()

#get the path to all csv files in the folder final_data_files
all_filenames = ['final_data_files/2_df_man.csv', 'final_data_files/3_df_man_PCA.csv', 'final_data_files/4_df_man_noNanCat.csv', 
                 'final_data_files/5_df_freq.csv', 'final_data_files/6_df_freq_PCA.csv', 'final_data_files/7_df_freq_noNanCat.csv']

results_rmse = []
results_mae = []
for i in range(len(all_filenames)):
    data_x = pd.read_csv(all_filenames[i], sep=',', index_col=0).drop(['Price'], axis=1)
    data_y = pd.read_csv(all_filenames[i], sep=',', index_col=0)['Price']

    rmse_list = []
    mae_list = []
    kFold = KFold(n_splits=5, shuffle=True, random_state= randomSeed)
    for train_index, test_index in kFold.split(data_x):
        X_train, X_test = data_x.iloc[train_index], data_x.iloc[test_index]
        y_train, y_test = data_y.iloc[train_index], data_y.iloc[test_index]

        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)

        model = Sequential()

        # Adding the input layer and the first hidden layer
        model.add(Dense(32, activation = 'relu', input_dim = X_train.shape[1]))

        # Adding the second hidden layer
        model.add(Dense(units = 32, activation = 'relu', activity_regularizer=regularizers.l2(0.01), 
                        kernel_regularizer=regularizers.l2(0.01)))

        # Add a dropout layer
        #model.add(Dropout(0.1))

        # Adding the third hidden layer
        model.add(Dense(units = 32, activation = 'relu', activity_regularizer=regularizers.l2(0.01), 
                        kernel_regularizer=regularizers.l2(0.01)))

        # Adding the output layer
        model.add(Dense(units = 1))

        #second best: lr= 0.01, batch_size = 10, epochs = 30, one dropout layer between 2 and 3
        # Compiling the ANN
        model.compile(optimizer = optimizers.Adam(learning_rate=0.01), loss = 'mean_squared_error')

        # Fitting the ANN to the Training set
        model.fit(X_train, y_train, batch_size = 10, epochs = 40)

        # Make predictions on the testing set
        predictions = model.predict(X_test)

        # Calculate and save the RMSE for this fold
        mse = mean_squared_error(y_test, predictions)
        rmse = np.sqrt(mse)
        rmse_list.append(rmse)

        # Calculate and save the MAE for this fold
        mae = mean_absolute_error(y_test, predictions)
        mae_list.append(mae)

    results_rmse.append(np.mean(rmse_list))
    results_mae.append(np.mean(mae_list))    


In [24]:
print(results_rmse)
print(results_mae)

[48295.01745793575, 48443.51129358997, 49463.290206533406, 63589.41982246747, 48277.07436660505, 53440.640553125704]
[28265.59456568516, 29015.693331531358, 28473.937515026133, 29326.47570930196, 28780.204483842168, 29235.305308654293]
